In [30]:
import pandas as pd


In [31]:
female = pd.read_csv('./female.csv')
male = pd.read_csv('./male.csv')

In [32]:
def compute_percentile_ranges(column):
    ranges = [(0,5),(5,25),(25,50),(50,75),(75,90),(90,97),(97,100)]
    percentiles = {(low,high):(round(column.quantile(low/100).item(), 2), round(column.quantile(high/100).item(), 2)) for low,high in ranges}

    counts = {}
    for r,(low,high) in percentiles.items():
        counts[r]= ((column >= low) & (column < high)).sum()
        
    return counts

   ## print(percentiles)

print(compute_percentile_ranges(female['chestcircumference']))
print(compute_percentile_ranges(female['biacromialbreadth']))

print(compute_percentile_ranges(male['chestcircumference']))
print(compute_percentile_ranges(male['biacromialbreadth']))




{(0, 5): np.int64(100), (5, 25): np.int64(396), (25, 50): np.int64(492), (50, 75): np.int64(499), (75, 90): np.int64(299), (90, 97): np.int64(140), (97, 100): np.int64(59)}
{(0, 5): np.int64(93), (5, 25): np.int64(377), (25, 50): np.int64(477), (50, 75): np.int64(541), (75, 90): np.int64(297), (90, 97): np.int64(139), (97, 100): np.int64(61)}
{(0, 5): np.int64(199), (5, 25): np.int64(810), (25, 50): np.int64(1025), (50, 75): np.int64(1012), (75, 90): np.int64(616), (90, 97): np.int64(295), (97, 100): np.int64(124)}
{(0, 5): np.int64(191), (5, 25): np.int64(787), (25, 50): np.int64(989), (50, 75): np.int64(1079), (75, 90): np.int64(610), (90, 97): np.int64(303), (97, 100): np.int64(122)}


In [33]:
def compute_size_percentile_mesurements(data,chest_column,shoulder_column):
    sizes = ['XS','S','M','L','XL','2XL','3XL']
    ranges = [0,5,25,50,75,90,97]

    #compute the values for each percentile for chest and shoulder
    
    chest_percentiles = {p:data[chest_column].quantile(p/100) for p in ranges}
    shoulder_percentiles = {p:data[shoulder_column].quantile(p/100) for p in ranges}
 #map the T-Shirt sizes to the corresponding chest and shoulder measurements
    size_mappings = {}
    for i, size in enumerate(sizes):
        size_mappings[size] = {
            'chest': int(chest_percentiles[ranges[i]]),
            'shoulder': int(shoulder_percentiles[ranges[i]])

        }
    return size_mappings
print(compute_size_percentile_mesurements(female,'chestcircumference','biacromialbreadth'))
print(compute_size_percentile_mesurements(male,'chestcircumference','biacromialbreadth'))




   # print(chest_percentiles)
    #print(shoulder_percentiles)
    
#compute_size_percentile_mesurements(female,'chestcircumference','biacromialbreadth')


{'XS': {'chest': 695, 'shoulder': 283}, 'S': {'chest': 824, 'shoulder': 335}, 'M': {'chest': 889, 'shoulder': 353}, 'L': {'chest': 940, 'shoulder': 365}, 'XL': {'chest': 999, 'shoulder': 378}, '2XL': {'chest': 1057, 'shoulder': 389}, '3XL': {'chest': 1117, 'shoulder': 400}}
{'XS': {'chest': 774, 'shoulder': 337}, 'S': {'chest': 922, 'shoulder': 384}, 'M': {'chest': 996, 'shoulder': 403}, 'L': {'chest': 1056, 'shoulder': 415}, 'XL': {'chest': 1117, 'shoulder': 428}, '2XL': {'chest': 1172, 'shoulder': 441}, '3XL': {'chest': 1233, 'shoulder': 452}}


In [38]:
female_sizes = { 'XS': {'chest': 695, 'shoulder': 283},
                 'S': {'chest': 824, 'shoulder': 335}, 
                 'M': {'chest': 889, 'shoulder': 353},
                 'L': {'chest': 940, 'shoulder': 365}, 
                 'XL': {'chest': 999, 'shoulder': 378}, 
                 '2XL': {'chest': 1057, 'shoulder': 389}, 
                 '3XL': {'chest': 1117, 'shoulder': 400}
                }
male_sizes = { 'XS': {'chest': 774, 'shoulder': 337},
               'S': {'chest': 922, 'shoulder': 384},
               'M': {'chest': 996, 'shoulder': 403}, 
               'L': {'chest': 1056, 'shoulder': 415}, 
               'XL': {'chest': 1117, 'shoulder': 428}, 
               '2XL': {'chest': 1172, 'shoulder': 441},
               '3XL': {'chest': 1233, 'shoulder': 452}
            }


In [39]:
def get_sizes(data,size_chart):
    matches = {size: 0 for size in size_chart.keys()}
    ties = 0

    for _, row in data.iterrows():
        possible_sizes = []
    
        for size, mesurements in size_chart.items():
            if (row['biacromialbreadth'] <= mesurements['shoulder'] and 
                row ['chestcircumference'] <= mesurements['chest']):
                possible_sizes.append(size)

        if len(possible_sizes) == 1:
            matches[possible_sizes[0]] += 1
        elif len(possible_sizes) > 1:
            ties += 1
    return matches, ties

In [40]:
female_matches,female_ties = get_sizes(female,female_sizes)
male_matches, male_ties = get_sizes(male,male_sizes)

print('female matches:',female_matches)
print('female ties:',female_ties)
print('male matches:', male_matches)
print('male ties:', male_ties)

female matches: {'XS': 0, 'S': 0, 'M': 0, 'L': 0, 'XL': 0, '2XL': 0, '3XL': 236}
female ties: 1642
male matches: {'XS': 0, 'S': 0, 'M': 0, 'L': 0, 'XL': 0, '2XL': 0, '3XL': 434}
male ties: 3437


In [ ]:
def create_overlapping_size_chart(original_chart):
    overlapping_chart = {}

    sizes = list(original_chart.keys())

    for i, sizes in enumerate(sizes):
        overlapping_chart[sizes] = {}
        if i == 0:
            overlapping_chart[sizes]['chest'] = [original_chart[sizes]['chest'], original_chart[sizes[i+1]['chest']]+5]
            overlapping_chart[sizes]['shoulder'] = [original_chart[sizes]['shoulder'], original_chart[sizes[i+1]['shoulder']]+5]
        elif i == len(sizes)-1:
            overlapping_chart[sizes]['chest'] = [original_chart[sizes]['chest']-5, original_chart[sizes]['chest']+1000]
            overlapping_chart[sizes]['shoulder'] = [original_chart[sizes]['shoulder']-5, original_chart[sizes]['shoulder']+1000]
        else:
            overlapping_chart[sizes]['chest'] = [original_chart[sizes]['chest']-5, original_chart[sizes[i+1]['chest']]+5]
             overlapping_chart[sizes]['shoulder'] = [original_chart[sizes]['shoulder']-5, original_chart[sizes[i+1]['shoulder']]+5]